<a href="https://colab.research.google.com/github/AurellGE/text-lstm-keras-improvement/blob/main/%5BOld_Code%5D_text_classification_LSTM_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df=pd.read_csv('/content/drive/My Drive/Materi Kuliah/Text Mining/Code/text_classification_DL/Emotion_classify_Data.csv')

In [ ]:
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [ ]:
df.shape

(5937, 2)

In [ ]:
df['Emotion'].value_counts()

,count
Emotion,
anger,2000
joy,2000
fear,1937


In [ ]:
#Encoding Categorical to Numeric of Gender and Ever_Married
label_encode={"Emotion": {"anger":0,"joy" :1,"fear":2}}
df=df.replace(label_encode)


In [ ]:
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,2
1,im so full of life i feel appalled,0
2,i sit here to write i start to dig out my feel...,2
3,ive been really angry with r and i feel like a...,1
4,i feel suspicious if there is no one outside l...,2


In [ ]:
def cleansing(df):
    df_clean=df.str.lower()
    df_clean=[re.sub(r"\d+","",i )for i in df_clean]
    df_clean=[re.sub(r'[^\w]', ' ', i)for i in df_clean]
    df_clean=[re.sub(r'\s+',' ',i)for i in df_clean]
    return df_clean


In [ ]:
df['clean_text']=cleansing(df['Comment'])

In [ ]:
df.head()

,Comment,Emotion,clean_text
0,i seriously hate one subject to death but now ...,2,i seriously hate one subject to death but now ...
1,im so full of life i feel appalled,0,im so full of life i feel appalled
2,i sit here to write i start to dig out my feel...,2,i sit here to write i start to dig out my feel...
3,ive been really angry with r and i feel like a...,1,ive been really angry with r and i feel like a...
4,i feel suspicious if there is no one outside l...,2,i feel suspicious if there is no one outside l...


In [ ]:
#check maximum length of word in sentence
max_sen = df['clean_text'].str.split().str.len().max()

In [ ]:
max_sen

64

**Split the Data**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, log_loss
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(df['clean_text'], df['Emotion'], test_size = 0.2, random_state = 42,stratify=df['Emotion'])

In [ ]:
len(x_train)

4749

In [ ]:
 x_train.str.split().str.len().max()

64

**Remove Stopword and Tokenization**

In [ ]:
#Define Stopwords
from nltk.corpus import stopwords
nltk.download('stopwords')
# Get a list of stop words in the English language
stop_words = set(stopwords.words('english'))

# Display the top 20 stop words
list(stop_words)[:20]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['isn',
 'has',
 'wasn',
 'are',
 'after',
 'him',
 'for',
 'mustn',
 'doing',
 'an',
 'to',
 'been',
 'again',
 "couldn't",
 "you'll",
 'itself',
 "should've",
 'needn',
 'below',
 'most']

In [ ]:
#tokenization
nltk.download('punkt_tab')

from nltk.tokenize import word_tokenize # Import word_tokenize
word_token=[word_tokenize(i) for i in x_train]

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [ ]:
word_token

[['i',
  'fear',
  'that',
  'other',
  'people',
  'ask',
  'me',
  'about',
  'my',
  'feelings',
  'i',
  'am',
  'most',
  'reluctant',
  'to',
  'talk',
  'about',
  'things'],
 ['ive',
  'been',
  'thinking',
  'about',
  'what',
  'it',
  'is',
  'that',
  'drives',
  'me',
  'not',
  'only',
  'with',
  'fashion',
  'as',
  'pretentious',
  'as',
  'this',
  'is',
  'gon',
  'na',
  'make',
  'me',
  'sound',
  'i',
  'am',
  'studying',
  'fashion',
  'design',
  'so',
  'i',
  'do',
  'feel',
  'its',
  'kinda',
  'vital',
  'to',
  'understand',
  'what',
  'im',
  'trying',
  'to',
  'do',
  'there',
  'but',
  'in',
  'life',
  'as',
  'a',
  'whole'],
 ['i', 'love', 'comments', 'so', 'feel', 'free'],
 ['i',
  'feel',
  'irritable',
  'like',
  'no',
  'other',
  'and',
  'running',
  'will',
  'def',
  'cure',
  'that'],
 ['i',
  'dont',
  'have',
  'a',
  'yeast',
  'infection',
  'in',
  'the',
  'vagina',
  'i',
  'could',
  'be',
  'feeling',
  'irritated',
  'by',
  

In [ ]:
len(word_token)

4749

In [ ]:
# Remove stopwords from each sublist in word_token
filtered_tokens_train = [[word for word in sublist if word not in stop_words] for sublist in word_token]

# Display the first sublist of filtered tokens after removing stopwords
print(filtered_tokens_train[0])

['fear', 'people', 'ask', 'feelings', 'reluctant', 'talk', 'things']


In [ ]:
print(filtered_tokens_train[0])

['ive', 'thinking', 'drives', 'fashion', 'pretentious', 'gon', 'na', 'make', 'sound', 'studying', 'fashion', 'design', 'feel', 'kinda', 'vital', 'understand', 'im', 'trying', 'life', 'whole']


**Vectorization Word2Vec - Skipgram**

In [ ]:
import gensim
from gensim.models import Word2Vec
model_skipgram = gensim.models.Word2Vec(filtered_tokens_train, min_count = 1,vector_size = 100, window = 3, sg=1)

In [ ]:
vocabulary_skipgram = model_skipgram.wv.index_to_key
word_vec_dict={}
for word in vocabulary_skipgram:
    word_vec_dict[word]=model_skipgram.wv.get_vector(word)
print("The no of key-value pairs : ",len(word_vec_dict)) # should come equal to vocab size


The no of key-value pairs :  7769


In [ ]:
word_vec_dict

{'feel': array([-0.07431346,  0.15219785,  0.15966883,  0.04915012,  0.02462526,
        -0.34054124,  0.07207248,  0.5436141 , -0.24471797, -0.26064682,
         0.00614684, -0.3965749 ,  0.00284699,  0.13873895,  0.12127721,
        -0.24649507,  0.1749292 , -0.1549097 , -0.18682104, -0.5508534 ,
         0.17282912,  0.11255416,  0.21137317, -0.10835481,  0.00652656,
        -0.10592979, -0.15350829, -0.0562281 , -0.31686884,  0.02714374,
         0.20269866, -0.00219727,  0.11950304, -0.2762311 , -0.21358272,
         0.1936684 ,  0.14815535, -0.24405712, -0.20897615, -0.35954762,
        -0.07132005, -0.1751184 , -0.11260797, -0.02468643,  0.22046696,
        -0.01130546, -0.03416028,  0.04430991,  0.04732836,  0.25974923,
         0.10975626, -0.14029852, -0.09485561, -0.07425831, -0.07748424,
         0.24735369,  0.06054443, -0.13437478, -0.16281494,  0.32239652,
         0.11768872,  0.06715538,  0.03097085, -0.00796831, -0.2923929 ,
         0.17164928, -0.01914866,  0.154153

In [ ]:
max_sen_len= max_sen # max lenght of word in a sentence
vocab_size =35000  #ideally it should be len(tok.word_index) + 1  or total no of words in data in this case = 4750, but to handle number of data which not appear in train, for example in test, make the size higher
embed_dim=100 # embedding dimension as choosen in word2vec constructor

In [ ]:
#!pip install tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer # Import Tokenizer from the correct module in TensorFlow
#from keras.preprocessing.text import one_hot,Tokenizer
tok = Tokenizer()
tok.fit_on_texts(filtered_tokens_train)
vocab_size = vocab_size
encd_rev = tok.texts_to_sequences(filtered_tokens_train)

In [ ]:
# now creating the embedding matrix
embed_matrix=np.zeros(shape=(vocab_size,embed_dim))
for word,i in tok.word_index.items():
    embed_vector=word_vec_dict.get(word) #mapping the vector to word in our skipgram dictionary
    if embed_vector is not None:  # word is in the vocabulary learned by the w2v model
        embed_matrix[i]=embed_vector
  # if word is not found then embed_vector corressponding to that vector will stay zero.

**Preparing the Data for Embedding Layer**

In [ ]:
encd_rev[0]

[566, 10, 281, 49, 117, 190, 25]

In [ ]:
vocab_size

35000

In [ ]:
from keras.preprocessing.sequence import pad_sequences
# now padding to have a maximum length of 64
pad_rev= pad_sequences(encd_rev, maxlen=max_sen_len, padding='post')
pad_rev.shape   # note that we had 4749 data and we have padded each review to have  a lenght of 64 words.

(4749, 64)

In [ ]:
pad_rev

array([[ 566,   10,  281, ...,    0,    0,    0],
       [  18,  178, 2211, ...,    0,    0,    0],
       [  31,  603,    1, ...,    0,    0,    0],
       ...,
       [ 693,  640,  151, ...,    0,    0,    0],
       [   1,    3, 7767, ...,    0,    0,    0],
       [  18,    2,  198, ...,    0,    0,    0]], dtype=int32)

In [ ]:
pad_rev[0] #that show index of word

array([566,  10, 281,  49, 117, 190,  25,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0],
      dtype=int32)

**Classification**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.initializers import Constant
from tensorflow.keras.layers import Dense, Embedding, Activation, Flatten

In [ ]:
model = keras.Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embed_dim,input_length=max_sen_len,embeddings_initializer=Constant(embed_matrix)))
model.add(layers.LSTM(64, input_shape=(None, 28)))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3))
print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer="sgd",
    metrics=["accuracy"],
)

In [ ]:
model.fit(pad_rev, y_train, batch_size=2, epochs=2)

Epoch 1/2
2375/2375 ━━━━━━━━━━━━━━━━━━━━ 81s 33ms/step - accuracy: 0.3189 - loss: 1.1021
Epoch 2/2
2375/2375 ━━━━━━━━━━━━━━━━━━━━ 82s 33ms/step - accuracy: 0.3235 - loss: 1.1012


In [ ]:
4749/2


**Testing**

**Preparing  Test Data for Embedding Layer**

In [ ]:
# Tokenize the words in the testing text data
word_token_test =[word_tokenize(i) for i in x_test]

In [ ]:
word_token_test[0]

['i',
 'sometimes',
 'feel',
 'shy',
 'about',
 'my',
 'musical',
 'taste',
 'because',
 'some',
 'of',
 'it',
 'wanders',
 'towards',
 'what',
 'some',
 'might',
 'call',
 'techno',
 'slander']

In [ ]:
# Remove stopwords from each sublist in word_token_test
filtered_tokens_test = [[word for word in sublist if word not in stop_words] for sublist in word_token_test]
# Display the first sublist of filtered tokens after removing stopwords
print(filtered_tokens_test[0])

['sometimes', 'feel', 'shy', 'musical', 'taste', 'wanders', 'towards', 'might', 'call', 'techno', 'slander']


In [ ]:
tok.fit_on_texts(filtered_tokens_test)
encd_rev_test = tok.texts_to_sequences(filtered_tokens_test)

In [ ]:
encd_rev_test

[[75, 1, 191, 2594, 1372, 7846, 229, 141, 289, 7847, 7848],
 [7849,
  3383,
  781,
  205,
  594,
  7850,
  1394,
  90,
  68,
  8,
  94,
  1680,
  26,
  1100,
  7851,
  140,
  299,
  1717,
  2008,
  41,
  5,
  593],
 [58, 1, 3, 1383, 266, 425, 150, 1175, 15, 2086, 3810],
 [2, 227, 659, 505, 91, 7852],
 [333, 2, 548, 336, 2548, 7853, 89, 2373, 7854, 2558, 7855, 3828, 1, 296],
 [6, 2545, 7856, 105, 8, 1, 192, 279, 2463, 419, 626],
 [893, 1655, 1, 607, 367, 434, 7857, 585, 7858, 1655],
 [311, 1, 714],
 [1, 37, 38, 1191, 54, 141, 882, 1, 217, 1488, 706, 49, 477, 835],
 [573, 2, 5, 511, 44, 884],
 [18, 2, 157, 108, 7859, 1802, 18, 1608, 33],
 [1, 1059, 1644, 283, 179],
 [1, 631, 353, 883, 214],
 [1, 134],
 [393, 1256, 273, 70, 2, 60, 104],
 [7860, 7861, 221, 25, 63, 1648, 1346, 7862, 53, 71, 1798, 1648, 1, 153],
 [1, 13, 210, 141, 2, 132, 1789, 27],
 [4, 432, 7863, 7864, 1, 4, 7865, 23, 303, 40],
 [1, 1015, 507, 1837, 681],
 [2, 520, 832],
 [4, 64, 1162, 2, 108],
 [2,
  293,
  108,
  172,
  

In [ ]:
from keras.preprocessing.sequence import pad_sequences
# now padding to have a maximum length of 64
pad_rev_test= pad_sequences(encd_rev_test, maxlen=max_sen_len, padding='post')
pad_rev_test.shape   # note that we had 10 testing data and we have padded each review to have  a lenght of 64 words.

(1188, 64)

In [ ]:
pad_rev_test

In [ ]:
test_predict=model.predict(pad_rev_test)
classe_test=np.argmax(test_predict,axis=1)


38/38 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step


In [ ]:
classe_test

array([1, 1, 1, ..., 1, 1, 1])

In [ ]:
print('\nClassification Report\n')
print(classification_report(y_test, classe_test))


Classification Report

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       400
           1       0.34      1.00      0.50       400
           2       0.00      0.00      0.00       388

    accuracy                           0.34      1188
   macro avg       0.11      0.33      0.17      1188
weighted avg       0.11      0.34      0.17      1188

